In [3]:
import folium #Umożliwia tworzenie interaktywnych map w języku Python
from IPython.display import display # Umożliwia wyświetlanie interaktywnych wyników w środowisku IPython
from geopy.geocoders import Nominatim #służy do geokodowania, czyli przekształcania adresów lub miejsc na współrzędne geograficzne.
import requests
import polyline # używana do reprezentacji tras na mapie.
import pandas as pd
from folium.plugins import MarkerCluster, FeatureGroupSubGroup

In [4]:
df = pd.read_csv('./sample4map2.csv', index_col=0)

In [5]:
df

,nazwa ogloszenia,cena,opis,lokalizacja,dzielnica,ulica,zametr,pokoje,powierzchnia,wysposażenie,...,typ,forma wlasnosci,stan wykonczenia,balkon/ogrod/taras,miejsce parkingowe,ogrzewanie,telefon,link,Score,ulica_dzielnica
82,2-pokojowe mieszkanie 38m2 + 2 ogródki,509 145 zł,Wirtualny spacer,"ul. Konińska, Leśnica, Fabryczna, Wrocław, dol...",Fabryczna,"Konińska, Wrocław",13 099 zł/m²,2.0,"38,87 m²",NaN,...,NaN,pełna własność,do wykończenia,"taras, ogródek",garaż/miejsce parkingowe,NaN,573 560 379,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,10.000000,Rudzka
14,PROMOCJA | NajlepszaCENA |SuperUstawne|Sprawdź...,539 000 zł,Pracujemy 7 dni w tygodniu. Zainteresowała Cię...,"ul. Ignacego Mościckiego, Bieńkowice, Krzyki, ...",Krzyki,"Ignacego Mościckiego, Wrocław",7 985 zł/m²,4.0,"67,5 m²",brak informacji,...,biuro nieruchomości,pełna własność,NaN,"balkon, taras",garaż/miejsce parkingowe,miejskie,690 725 555,https://www.otodom.pl/pl/oferta/promocja-najle...,8.154165,Gajowa
62,2-pokojowe mieszkanie 40m2 + loggia Bez Prowizji,522 329 zł,Wirtualny spacer,"ul. Konińska, Leśnica, Fabryczna, Wrocław, dol...",Fabryczna,"Konińska, Wrocław",12 768 zł/m²,2.0,"40,91 m²",NaN,...,NaN,pełna własność,do wykończenia,NaN,garaż/miejsce parkingowe,NaN,573 560 379,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,6.260087,Rudzka
96,OSTATNIE|NajlepszaCena|promocja|NowaInwestycja...,469 000 zł,Pracujemy 7 dni w tygodniu. Zainteresowała Cię...,"ul. Ignacego Mościckiego, Bieńkowice, Krzyki, ...",Krzyki,"Ignacego Mościckiego, Wrocław",8 058 zł/m²,3.0,"58,2 m²",brak informacji,...,biuro nieruchomości,pełna własność,NaN,"balkon, taras",garaż/miejsce parkingowe,miejskie,690 725 555,https://www.otodom.pl/pl/oferta/ostatnie-najle...,5.312112,Gajowa
60,"Okazja 2 pok , Centrum, Wrocław",403 000 zł,Sprzedam mieszkanie 37m2 we Wrocławiu. Mieszka...,"ul. Nowowiejska, Ołbin, Śródmieście, Wrocław, ...",Śródmieście,"Nowowiejska, Wrocław",10 605 zł/m²,2.0,38 m²,"piekarnik, kuchenka, pralka",...,biuro nieruchomości,NaN,do zamieszkania,NaN,garaż/miejsce parkingowe,NaN,536 186 831,https://www.otodom.pl/pl/oferta/okazja-2-pok-c...,4.643208,Kossaka
110,Ostatnie| Nowe Budownictwo | Balkon | Obejrzyj...,453 000 zł,Pracujemy 7 dni w tygodniu. Zainteresowała Cię...,"ul. Woskowa, Brochów, Krzyki, Wrocław, dolnośl...",Krzyki,"Woskowa, Wrocław",9 867 zł/m²,3.0,"45,91 m²",brak informacji,...,biuro nieruchomości,pełna własność,NaN,"balkon, taras",garaż/miejsce parkingowe,miejskie,690 725 555,https://www.otodom.pl/pl/oferta/ostatnie-nowe-...,2.792873,Gajowa
29,OSTATNIE NAROŻNE|Garderoba| DużeOkna| Sprawdź|...,489 000 zł,Pracujemy 7 dni w tygodniu. Zainteresowała Cię...,"ul. Woskowa, Brochów, Krzyki, Wrocław, dolnośl...",Krzyki,"Woskowa, Wrocław",9 839 zł/m²,3.0,"49,7 m²",brak informacji,...,biuro nieruchomości,pełna własność,NaN,"balkon, taras",garaż/miejsce parkingowe,miejskie,690 725 555,https://www.otodom.pl/pl/oferta/ostatnie-naroz...,2.489565,Gajowa
71,Okolice ul. Żelaznej Inwestycja/COmiejskie,518 000 zł,"Na sprzedaż dwupokojowe, zadbane oraz odświeżo...","Gajowice, Fabryczna, Wrocław, dolnośląskie",Fabryczna,"Rudzka, Wrocław",14 153 zł/m²,2.0,"36,6 m²",brak informacji,...,biuro nieruchomości,pełna własność,NaN,balkon,NaN,miejskie,698 666 308,https://www.otodom.pl/pl/oferta/okolice-ul-zel...,2.456037,Rudzka
7,PROMOCJA | Super Ustawne | Nowa Inwestycja | HIT!,488 400 zł,Pracujemy 7 dni w tygodniu. Zainteresowała Cię...,"ul. Woskowa, Brochów, Krzyki, Wrocław, dolnośl...",Krzyki,"Woskowa, Wrocław",9 947 zł/m²,3.0,"49,1 m²",brak informacji,...,biuro nieruchomości,pełna własność,NaN,"balkon, taras",garaż/miejsce parkingowe,miejskie,690 725 555,https://www.otodom.pl/pl/oferta/promocja-super...,2.335822,Gajowa
112,PROMOCJA | NajlepszaCENA | OstatnieSztuki | HIT!,329 000 zł,Pracujemy 7 dni w tygodniu. Zainteresowała Cię...,"ul. Ignacego Mościckiego, Bieńkowice, Krzyki, ...",Krzyki,"Ignacego Mościckiego, Wrocław",9 014 zł/m²,2.0,"36,5 m²",brak informacji,...,biuro nieruchomości,pełna 

In [6]:
def rysowanie_mapki(df):
    '''Funkcja przyjmująca df, ktory musi zawierac kolumne ulice oraz Score. IM jako punkt startowy jest dopisywany automatycznie.
    Df w srodku sortuje sie wzgledem punktacji i ustawia kolorki ikonki itd odpowiednie dla oferty. 

    Zwraca liste w której pierwszy element to lista tekstów dotyczacych długosci podrozy, a drugi element to
    dfmapke z ofertami, ktore zostaly uzyte. Mapka wyświetla się automatycznie i tworzy plik html w miejscu gdzie jest plik z kodem, na mapie mamy
    sciezke miedzy glownymi ofertami + dolozone sa oferty ktore sa 'po drodze'. '''
    
    df=pd.concat([df,pd.DataFrame(data=[['plac Grunwaldzki 2, Wrocław',999]],columns=['ulica','Score'])],ignore_index=True)  #dodanie im jako startu
    kolorki = ['blue','green','green','green','orange','orange','orange','orange']
    ikonki = ['search','home','home','home','map-marker','map-marker','map-marker','map-marker']
    df=df.sort_values(by='Score',ascending=False)
    df=df[0:8]
    df['kolor']=kolorki
    df['ikona']=ikonki
    wykorzystane=df[1:4]
    mapbox_api_key = ''

    geolocator = Nominatim(user_agent="my_geocoder") #definiujemy usługę geokodowania
    
    loc = geolocator.geocode('Wrocław')
    loc = [loc.latitude,loc.longitude]
    
    map_wroclaw = folium.Map(loc, zoom_start=12, control_scale=True)
    # Dodawanie warstwy mapbox do mapy
    folium.TileLayer(
        tiles=f'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}',
        attr='Mapbox',
        name='Mapbox Streets',
        overlay=True
    ).add_to(map_wroclaw)
    

    coordinates=[]
    adres = []
    	
    marker_cluster = MarkerCluster().add_to(map_wroclaw)
    
    for i in range(0,8): #stawianie 3 glownych punktow i IM, i zbieranie koordynatow wszystkich
        adres.append(df.iloc[i]['ulica'])
        kolor = df.iloc[i]['kolor']
        ikona = df.iloc[i]['ikona']
        location = geolocator.geocode(adres[i])
        if i <= 3: #zeby na razie dodalo tylko im i 3 glowne punkty
            folium.Marker(location=[location.latitude, location.longitude], popup=adres[i],icon=folium.Icon(color=kolor, icon=ikona)).add_to(marker_cluster)
        coordinates.append([location.longitude,location.latitude])
    
    
    #szukanie optymalnej drogi miedzy im a trzema najlepszymi
    profile = 'driving'
    optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]}?access_token={mapbox_api_key}'
    response = requests.get(optimized_trips_url)
    data_opt = response.json()
    dystans = round(data_opt['trips'][0]['distance']/1000,2)
    czas = round(data_opt['trips'][0]['duration']/60)
    #ponizej sprawdzamy dla pozostalych 4 opcjonlanych miejsc czy dolozenie ich 
    #zwiekszy czas przejazdu o poonad 20% - jesli tak olewamy, jesli nie dokladamy.
    czas_dodatkowy=[]
    for i in range(4,8):
        profile = 'driving'
        optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]};{coordinates[i][0]},{coordinates[i][1]}?access_token={mapbox_api_key}'
        response = requests.get(optimized_trips_url)
        data_opt = response.json()
        if data_opt['trips'][0]['duration']/60 <= czas*1.2:
            czas_dodatkowy.append([data_opt['trips'][0]['duration']/60 - czas,i])
    #sprawdzanie ile znalazlo i tworzenie z nimi nowej drogi
    #wpisywanie wspolrzednych do stringa i wrzucanie znacznikow dodatkowych miejsc na mape
    pozostale = '' 
    for i in range (0,len(czas_dodatkowy)):
        wykorzystane=pd.concat([wykorzystane,df.iloc[czas_dodatkowy[i][1]].to_frame().T],ignore_index=True) 
        pozostale = pozostale + ';' +str(coordinates[czas_dodatkowy[i][1]][0])+','+str(coordinates[czas_dodatkowy[i][1]][1])
        folium.Marker(location=[coordinates[czas_dodatkowy[i][1]][1], coordinates[czas_dodatkowy[i][1]][0]],popup=adres[czas_dodatkowy[i][1]],icon=folium.Icon(color='orange', icon='map-marker')).add_to(marker_cluster)
    
    #oficjalna trasa z punktami ktore sie oplaca
    optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]}{pozostale}?access_token={mapbox_api_key}'
    
    #reszta rzeczy zeby mapa sie pojawila i stringi z informacja ile sie jedzie bez / z punktami dodatkowymi.
    response = requests.get(optimized_trips_url)
    data_opt = response.json()
    coor = polyline.decode(data_opt['trips'][0]['geometry'])
    folium.PolyLine(locations=coor, color='blue').add_to(map_wroclaw)
    teksty = [
    f"Odległość trasy bez dodatkowych ofert: ok. {dystans} km",
    f"Czas podróży bez dodatkowych ofert: ok. {czas} min",
    f"Odległość trasy: ok. {round(data_opt['trips'][0]['distance']/1000,2)} km",
    f"Czas podróży: ok. {round(data_opt['trips'][0]['duration']/60)} min"
    ]
    
    
    map_wroclaw.save('map.html')
    display(map_wroclaw)
    return [teksty,wykorzystane]

In [7]:
rys=rysowanie_mapki(df)

In [8]:
rys

[['Odległość trasy bez dodatkowych ofert: ok. 60.48 km',
  'Czas podróży bez dodatkowych ofert: ok. 98 min',
  'Odległość trasy: ok. 61.32 km',
  'Czas podróży: ok. 104 min'],
                                     nazwa ogloszenia        cena  \
 0             2-pokojowe mieszkanie 38m2 + 2 ogródki  509 145 zł   
 1  PROMOCJA | NajlepszaCENA |SuperUstawne|Sprawdź...  539 000 zł   
 2   2-pokojowe mieszkanie 40m2 + loggia Bez Prowizji  522 329 zł   
 3  OSTATNIE|NajlepszaCena|promocja|NowaInwestycja...  469 000 zł   
 4                    Okazja 2 pok , Centrum, Wrocław  403 000 zł   
 5  Ostatnie| Nowe Budownictwo | Balkon | Obejrzyj...  453 000 zł   
 6  OSTATNIE NAROŻNE|Garderoba| DużeOkna| Sprawdź|...  489 000 zł   
 
                                                 opis  \
 0                                   Wirtualny spacer   
 1  Pracujemy 7 dni w tygodniu. Zainteresowała Cię...   
 2                                   Wirtualny spacer   
 3  Pracujemy 7 dni w tygodniu. Zaintereso